In [1]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, minmax_scale
from datetime import datetime

sys.path.append('../../library/')

from data import open_dataset, load_scaled_datasets, scale_images

In [2]:
data_path = "/glade/p/cisl/aiml/ai4ess_hackathon/holodec/"
num_particles = 3
split = "train"
output_cols = ["x", "y", "z", "d", "hid"]
input_scaler = MinMaxScaler()

In [ ]:
scaled_train_inputs, \
scaled_valid_inputs, \
scaled_train_outputs, \
scaled_valid_outputs, \
input_scaler = load_scaled_datasets(data_path,
                                    num_particles,
                                    output_cols,
                                    input_scaler)

In [3]:
model_path = "/glade/p/cisl/aiml/ggantos/holodec/"
models = [x[0] for x in os.walk(model_path)][1:]
models

['/glade/p/cisl/aiml/ggantos/holodec/conv2d_3particle',
 '/glade/p/cisl/aiml/ggantos/holodec/conv2d_1particle_test',
 '/glade/p/cisl/aiml/ggantos/holodec/conv2d_3particle_middle',
 '/glade/p/cisl/aiml/ggantos/holodec/conv2d_3particle_bigger',
 '/glade/p/cisl/aiml/ggantos/holodec/conv2d_1particle',
 '/glade/p/cisl/aiml/ggantos/holodec/conv2d_3particle_test',
 '/glade/p/cisl/aiml/ggantos/holodec/conv2d_1particle_20']

In [4]:
! ls /glade/p/cisl/aiml/ggantos/holodec/conv2d_3particle_test

cnn.h5	    pred_train_outputs.csv	   scaled_pred_valid_outputs.csv
config.yml  pred_valid_outputs.csv
error.csv   scaled_pred_train_outputs.csv


In [5]:
train_outputs = open_dataset(data_path, num_particles, "train")[output_cols].to_dataframe()
valid_outputs = open_dataset(data_path, num_particles, "valid")[output_cols].to_dataframe()
if len(output_cols) == 5:
    train_outputs = flatten_dataset(train_outputs)
    valid_outputs = flatten_dataset(valid_outputs)

In [6]:
model = models[0]
# error = pd.read_csv(os.path.join(model, 'error.csv'))
pred_train_outputs = pd.read_csv(os.path.join(model, 'pred_train_outputs.csv'))
pred_valid_outputs = pd.read_csv(os.path.join(model, 'pred_valid_outputs.csv'))
scaled_pred_train_outputs = pd.read_csv(os.path.join(model, 'scaled_pred_train_outputs.csv'))
scaled_pred_valid_outputs = pd.read_csv(os.path.join(model, 'scaled_pred_valid_outputs.csv'))

In [9]:
pred_valid_outputs.columns

Index(['index', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'], dtype='object')

In [11]:
error = {"train": {}, "valid": {}}
for i, var in enumerate(train_outputs.columns):
    err = mean_absolute_error(train_outputs[var], pred_train_outputs[str(i)])
    error["train"][var] = err
    print (f"Training error in {var}: ", err)
for i, var in enumerate(valid_outputs.columns):
    err = mean_absolute_error(valid_outputs[var], pred_valid_outputs[str(i)])
    error["valid"][var] = err
    print (f"Validation error in {var}: ", err)    

Training error in x1:  316.52819556925556
Training error in y1:  210.96780811121016
Training error in z1:  27083.941061460653
Training error in d1:  9.39672404200442
Training error in x2:  317.9517007582583
Training error in y2:  211.0155743174734
Training error in z2:  25877.62634116
Training error in d2:  9.175959639524848
Training error in x3:  318.52826259454747
Training error in y3:  213.5020698560284
Training error in z3:  26712.795618144202
Training error in d3:  9.35161267816209
Validation error in x1:  313.25150140775014
Validation error in y1:  216.95434742877424
Validation error in z1:  28872.224312064845
Validation error in d1:  9.78174555984602
Validation error in x2:  325.53328065126595
Validation error in y2:  212.56501547349467
Validation error in z2:  28849.615738700908
Validation error in d2:  9.746923511759583
Validation error in x3:  321.2350278685548
Validation error in y3:  212.42184387061295
Validation error in z3:  28580.104557069586
Validation error in d3:  9.7

In [ ]:
error